In [1]:
import pandas as pd
import sklearn
import statsmodels.api as sm

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# wrapper for statsmodels https://stackoverflow.com/questions/41045752/
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [3]:
def sklearn_cv(X, y, cv=None):
    cv_results = cross_val_score(
        SMWrapper(sm.OLS), 
        X, 
        y, 
        cv=cv, 
        scoring='neg_mean_squared_error'
    )
    neg_mse = cv_results.mean()
    return neg_mse
    
def islr_loocv(X, y):
    linear_model = sm.OLS(endog=y, exog=X)
    results = linear_model.fit()

    residuals = results.resid
    leverage = results.get_influence().hat_diag_factor
    loocv_mse = ((residuals / (1 - leverage)) ** 2).mean() 
    return loocv_mse

In [4]:
auto_data = pd.read_csv('../data/auto.csv')

In [5]:
polynomial_features = PolynomialFeatures(degree=2)
X = auto_data['horsepower'].to_numpy().reshape(-1, 1)
X = polynomial_features.fit_transform(X) # for quadratic fit on auto data set
y = auto_data['mpg'].to_numpy().reshape(-1, 1)

In [6]:
# LOOCV using sklearn - fit model n times
loocv = LeaveOneOut()
sklearn_loocv_mse = sklearn_cv(X, y, cv=loocv)
print(f'Sklearn LOOCV MSE: {sklearn_loocv_mse}')

Sklearn LOOCV MSE: -19.248213124489535


In [7]:
# LOOCV using ISLR magic formula - fit model once
islr_loocv_mse = islr_loocv(X, y)
print(f'ISLR LOOCV MSE: {islr_loocv_mse}')

ISLR LOOCV MSE: 19.25171903816222


In [8]:
# k-fold cv using sklearn - fit model k times
num_folds = [5, 10, 25, 50]
for num_fold in num_folds:
    kfold_mse = sklearn_cv(X, y, cv=num_fold)
    print(f'{num_fold}-fold MSE: {kfold_mse}')

5-fold MSE: -24.34715884368145
10-fold MSE: -21.23584005580647
25-fold MSE: -20.328175210303556
50-fold MSE: -19.89226257907138


In [9]:
%%timeit 
sklearn_cv(X, y, cv=loocv)

420 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
islr_loocv(X, y)

431 µs ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit 
sklearn_cv(X, y, cv=100) # 100 fold

118 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
